# Notebook to Explore Toronto Neighborhood

### installation of Beautiful Soup package

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

In [2]:
from bs4 import BeautifulSoup

## Use BeautifulSoup to Parse the website of List of postal codes of Canada: M

In [3]:
import requests
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(response.text, 'html.parser')

## get the table of postal code

In [4]:
table = soup.find('table')
table_row = table.find_all('tr')
row_all = []
for tr in table_row:
    td = tr.find_all('td')
    row = [i.text.rstrip('\n') for i in td]
    row_all.append(row)

## transfer the list into a dataframe

In [5]:
import pandas as pd
df = pd.DataFrame(row_all[1:])
df.columns=['PostalCode','Borough','Neighborhood']

In [6]:
df2 = df[df['Borough'].isin(['Not assigned'])==False]
df2.index=range(len(df2))
df2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## dimension of the dataframe

In [7]:
df2.shape

(103, 3)

## get the lat, long for each postal codes

### I used the "Geospatial_Coodinates.csv" file

In [8]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge two dataframe

In [9]:
df3=df2.copy()
df_inner = pd.merge(df3, dff, on='PostalCode', how='inner')
df_inner.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
dft = df_inner.copy()
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dft['Borough'].unique()),
        dft.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


## Explore the Neighborhoods of Toronto

In [11]:
address = 'Toronto, Ontario'
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Toronto map with the Neighborhood

In [13]:
# create map of Toronto using latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes #
import folium
map_tr = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dft['Latitude'], dft['Longitude'], dft['Borough'], dft['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tr)  
    
map_tr

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

## Explore the Borough of "North Park" - one of the bororgh in Toronto

In [14]:
North_data = dft[dft['Borough'] == 'North York'].reset_index(drop=True)
North_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [15]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York, Toronto are 43.7543263, -79.44911696639593.


In [16]:
map_North = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(North_data['Latitude'], North_data['Longitude'], North_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_North)  
    
map_North

## preparation to use FourSquare to explore the Venues around North Park

In [17]:
CLIENT_ID = 'O22FUJMMFHZYPFQ3IVCCEK5XFFTDSZGR0YDRSXHD4X01ZOPL' # your Foursquare ID
CLIENT_SECRET = 'TTUS3HGOT0BGINEUAIA2LUY3LQ0ZTCWCFBL14GENOA1N1SME' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O22FUJMMFHZYPFQ3IVCCEK5XFFTDSZGR0YDRSXHD4X01ZOPL
CLIENT_SECRET:TTUS3HGOT0BGINEUAIA2LUY3LQ0ZTCWCFBL14GENOA1N1SME


In [18]:
North_data.loc[0, 'Neighborhood']

'Parkwoods'

In [19]:
neighborhood_latitude = North_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = North_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = North_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [20]:
# url
search_query = 'Parkwoods'
LIMIT = 100
radius = 5000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=O22FUJMMFHZYPFQ3IVCCEK5XFFTDSZGR0YDRSXHD4X01ZOPL&client_secret=TTUS3HGOT0BGINEUAIA2LUY3LQ0ZTCWCFBL14GENOA1N1SME&ll=43.7532586,-79.3296565&v=20180605&query=Parkwoods&radius=5000&limit=100'

## get json file from FourSquare call

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edfdf1cc7638759c3a349fa'},
 'response': {'venues': [{'id': '4d1429d78f4c60fccab8ff1a',
    'name': 'Parkwoods',
    'location': {'lat': 43.749102676061796,
     'lng': -79.32216882705688,
     'distance': 759,
     'cc': 'CA',
     'city': 'North York',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['North York ON', 'Canada']},
    'categories': [{'id': '4f2a25ac4b909258e854f55f',
      'name': 'Neighborhood',
      'pluralName': 'Neighborhoods',
      'shortName': 'Neighborhood',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591729970',
    'hasPerk': False},
   {'id': '4e0bc2a8b61c18a1dc1e3898',
    'name': '67 Parkwoods Village Drive',
    'location': {'address': '67 Parkwoods Village Dr.',
     'lat': 43.7604317463954,
     'lng': -79.32213841379497,
     'labeledLatLngs': [{'label': 'display',
    

In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## nearby venues

In [24]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe    

venues = results['response']['venues'] 
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head()

,name,categories,location.lat,location.lng
0,Parkwoods,Neighborhood,43.749103,-79.322169
1,67 Parkwoods Village Drive,Residential Building (Apartment / Condo),43.760432,-79.322138
2,Parkwoods Village Centre,Shopping Mall,43.760735,-79.324873
3,Parkwoods Hairstyling,Salon / Barbershop,43.760782,-79.324653
4,Parkwoods Coin Laundry,Laundry Service,43.760386,-79.324894


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']  #why this code works? 06062020
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
North_venues = getNearbyVenues(names=North_data['Neighborhood'],
                                   latitudes=North_data['Latitude'],
                                   longitudes=North_data['Longitude']
                                  )


Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [28]:
print(North_venues.shape)
North_venues.head()

(244, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [29]:
North_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Don Mills,25,25,25,25,25,25
Downsview,16,16,16,16,16,16
"Fairview, Henry Farm, Oriole",58,58,58,58,58,58
Glencairn,4,4,4,4,4,4
Hillcrest Village,4,4,4,4,4,4
Humber Summit,1,1,1,1,1,1


In [30]:
print('There are {} uniques categories.'.format(len(North_venues['Venue Category'].unique())))

There are 104 uniques categories.


In [31]:
# one hot encoding
North_onehot = pd.get_dummies(North_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_onehot['Neighborhood'] = North_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_onehot.columns[-1]] + list(North_onehot.columns[:-1])
North_onehot = North_onehot[fixed_columns]

North_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
North_onehot.shape

(244, 105)

In [33]:
North_grouped = North_onehot.groupby('Neighborhood').mean().reset_index()
North_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.086957,0.000000,0.000000,0.0,0.00,0.00,0.0000,0.043478,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.086957,0.00,0.000000,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.043478,0.043478,0.000000,0.043478,0.000000,0.00,0.000000,0.043478,0.00,0.0000,0.000000,0.0000,0.000000,0.043478,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.043478,0.0000,0.043478,0.000000,0.043478,0.043478,0.043478,0.043478,0.000000,0.00,0.000000,0.00,0.000000,0.043478,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.043478,0.043478,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0000
1,Bayview Village,0.000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.00,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0000
2,"Bedford Park, Lawrence Manor East",0.000,0.0000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0400,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.040000,0.0,0.040000,0.000000,0.000000,0.000000,0.000000,0.080000,0.04,0.000000,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.040000,0.040000,0.00,0.0000,0.000000,0.0000,0.000000,0.000000,0.04,0.000000,0.000000,0.08,0.000000,0.000000,0.040000,0.040000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.040000,0.040000,0.000000,0.00,0.000000,0.04,0.000000,0.080000,0.000000,0.

In [34]:
North_grouped.shape

(20, 105)

## get the top 5 venues

In [35]:
num_top_venues = 5

for hood in North_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = North_grouped[North_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop  0.09
1                 Bank  0.09
2       Sandwich Place  0.04
3   Frozen Yogurt Shop  0.04
4  Fried Chicken Joint  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Café  0.25
3                 Bank  0.25
4        Jewelry Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0           Sandwich Place  0.08
1       Italian Restaurant  0.08
2              Coffee Shop  0.08
3               Restaurant  0.08
4  Comfort Food Restaurant  0.04


----Don Mills----
              venue  freq
0               Gym  0.08
1        Restaurant  0.08
2  Asian Restaurant  0.08
3       Coffee Shop  0.08
4        Beer Store  0.08


----Downsview----
            venue  freq
0   Grocery Store  0.19
1            Park  0.12
2     Coffee Shop  0.06
3  Baseball Field  0.06
4    Liquor 

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
North_venues_sorted = pd.DataFrame(columns=columns)
North_venues_sorted['Neighborhood'] = North_grouped['Neighborhood']

for ind in np.arange(North_grouped.shape[0]):
    North_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_grouped.iloc[ind, :], num_top_venues)

North_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Mobile Phone Shop,Pet Store,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Grocery Store
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Diner,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Restaurant,Coffee Shop,Sandwich Place,Juice Bar,Liquor Store,Café,Indian Restaurant,Comfort Food Restaurant,Cupcake Shop
3,Don Mills,Coffee Shop,Japanese Restaurant,Asian Restaurant,Gym,Restaurant,Beer Store,Caribbean Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant
4,Downsview,Grocery Store,Park,Food Truck,Baseball Field,Liquor Store,Business Service,Home Service,Gym / Fitness Center,Shopping Mall,Coffee Shop


## Cluster Neighborhoods

In [38]:
# set number of clusters
kclusters = 5

North_grouped_clustering = North_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 4], dtype=int32)

In [39]:
North_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

North_merged = North_data
North_merged = North_merged.join(North_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

North_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Construction & Landscaping,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,French Restaurant,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Dessert Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Accessories Store,Women's Store,Miscellaneous Shop,Vietnamese Restaurant,Coffee Shop,Gift Shop,Event Space,Boutique
3,M3B,North York,Don Mills,43.745906,-79.352188,0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Gym,Restaurant,Beer Store,Caribbean Restaurant,Sandwich Place,Clothing Store,Chinese Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,0,Park,Pizza Place,Japanese Restaurant,Pub,Women's Store,Chocolate Shop,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


## Visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(North_merged['Latitude'], North_merged['Longitude'], North_merged['Neighborhood'], North_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## end of the code